# Research Analyzer - Multi-Analyst News Research System

This notebook implements a LangGraph-based research system that creates a team of specialized news analysts to conduct research on any topic and synthesize their findings into a comprehensive report.

## Phase 1: Setup & Environment

In this phase, we install dependencies and configure API keys.

In [ ]:
# Install required dependencies
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python python-dotenv pydantic

In [ ]:
import os
import getpass

def _set_env(var: str):
    """Set environment variable from user input if not already set."""
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set OpenAI API Key
_set_env("OPENAI_API_KEY")

In [ ]:
# Set Tavily API Key for web search
_set_env("TAVILY_API_KEY")

In [ ]:
# Initialize the Language Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("✓ LLM initialized successfully!")